In [ ]:
cicle_companie ={
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "type": "Polygon",
        "coordinates": [
          [
            [
              2.1879724143280446,
              41.41201892582043
            ],
            [
              2.1869823680848857,
              41.4119824445129
            ],
            [
              2.1860018598371798,
              41.41187335204748
            ],
            [
              2.1850403355970442,
              41.41169269940623
            ],
            [
              2.184107058300675,
              41.4114422269607
            ],
            [
              2.1832110184884743,
              41.41112434768855
            ],
            [
              2.1823608476224448,
              41.410742123904846
            ],
            [
              2.1815647348794336,
              41.410299237733255
            ],
            [
              2.180830348224681,
              41.409799955602864
            ],
            [
              2.1801647605280876,
              41.4092490871143
            ],
            [
              2.179574381436162,
              41.40865193867292
            ],
            [
              2.1790648956562295,
              41.408014262337566
            ],
            [
              2.1786412082467863,
              41.40734220037924
            ],
            [
              2.178307397439424,
              41.40664222608535
            ],
            [
              2.1780666754443723,
              41.40592108138155
            ],
            [
              2.1779213576139917,
              41.40518571187282
            ],
            [
              2.177872840257381,
              41.40444319993094
            ],
            [
              2.1779215873154345,
              41.4037006964736
            ],
            [
              2.178067126019961,
              41.4029653520924
            ],
            [
              2.1783080515737923,
              41.40224424819351
            ],
            [
              2.1786420408019187,
              41.401544328813806
            ],
            [
              2.1790658746375007,
              41.40087233376858
            ],
            [
              2.1795754692218714,
              41.400234733773864
            ],
            [
              2.180165915315243,
              41.39963766816689
            ],
            [
              2.180831525635459,
              41.39908688582323
            ],
            [
              2.1815658896665986,
              41.398587689837754
            ],
            [
              2.182361935408174,
              41.398144886500575
            ],
            [
              2.18321199746977,
              41.39776273905749
            ],
            [
              2.1841078908558336,
              41.39744492669847
            ],
            [
              2.185040989731437,
              41.39719450916711
            ],
            [
              2.186002310412787,
              41.39701389733077
            ],
            [
              2.1869825977863386,
              41.3969048299929
            ],
            [
              2.1879724143280446,
              41.3968683571699
            ],
            [
              2.18896223086975,
              41.3969048299929
            ],
            [
              2.189942518243302,
              41.39701389733077
            ],
            [
              2.1909038389246516,
              41.39719450916711
            ],
            [
              2.191836937800255,
              41.39744492669847
            ],
            [
              2.192732831186319,
              41.39776273905749
            ],
            [
              2.193582893247915,
              41.398144886500575
            ],
            [
              2.19437893898949,
              41.398587689837754
            ],
            [
              2.19511330302063,
              41.39908688582323
            ],
            [
              2.195778913340846,
              41.39963766816689
            ],
            [
              2.1963693594342173,
              41.400234733773864
            ],
            [
              2.196878954018588,
              41.40087233376858
            ],
            [
              2.19730278785417,
              41.401544328813806
            ],
            [
              2.197636777082297,
              41.40224424819351
            ],
            [
              2.1978777026361276,
              41.4029653520924
            ],
            [
              2.198023241340654,
              41.4037006964736
            ],
            [
              2.198071988398708,
              41.40444319993094
            ],
            [
              2.198023471042097,
              41.40518571187282
            ],
            [
              2.1978781532117164,
              41.40592108138155
            ],
            [
              2.197637431216665,
              41.40664222608535
            ],
            [
              2.1973036204093024,
              41.40734220037924
            ],
            [
              2.196879932999859,
              41.408014262337566
            ],
            [
              2.1963704472199272,
              41.40865193867292
            ],
            [
              2.195780068128001,
              41.4092490871143
            ],
            [
              2.1951144804314078,
              41.409799955602864
            ],
            [
              2.194380093776655,
              41.410299237733255
            ],
            [
              2.193583981033644,
              41.410742123904846
            ],
            [
              2.1927338101676144,
              41.41112434768855
            ],
            [
              2.1918377703554137,
              41.4114422269607
            ],
            [
              2.1909044930590444,
              41.41169269940623
            ],
            [
              2.1899429688189094,
              41.41187335204748
            ],
            [
              2.188962460571203,
              41.4119824445129
            ],
            [
              2.1879724143280446,
              41.41201892582043
            ]
          ]
        ]
      }
    }
  ]
}

square_companie = {
  "type": "FeatureCollection",
  "features": [
    {
      "type": "Feature",
      "properties": {},
      "geometry": {
        "coordinates": [
          [
            [
              2.1788855567631913,
              41.40277234304929
            ],
            [
              2.188765073647744,
              41.39512215231312
            ],
            [
              2.1979086358985,
              41.40298720629434
            ],
            [
              2.188509782046765,
              41.40991467900736
            ],
            [
              2.1788855567631913,
              41.40277234304929
            ]
          ]
        ],
        "type": "Polygon"
      }
    }
  ]
}




[2.17,41.40],
[2.18,41.39],
[2.19,41.40],
[2.18,41.40],
[2.17,41.40]